# Card Classifer using Pytorch

In [ ]:
import sys
import timm
import torch
import torchvision
import numpy as np
import pandas as pd
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder

print(f"System Version: {sys.version}")
print(f"Pytorch Version: {torch.__version__}")
print(f"TorchVision Version: {torchvision.__version__}")
print(f"Numpy Version: {np.__version__}")
print(f"Pandas Version: {pd.__version__}")

# Create CardDataset class instance

In [ ]:
class CardDataset(Dataset):
    def __init__(self, root, transform=None):
        self.data = ImageFolder(root=root, transform=transform)
        pass

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        return self.data[index]

    @property
    def classes(self):
        return self.data.classes

# Load the dataset in from data/ directory

In [ ]:
dataset = CardDataset(root="./data/train")
print(f"Size: {len(dataset)}")

In [ ]:
image, label = dataset[6000]

image